In [ ]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup as bs
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


# Task: Columns -> Energy & Petrol Consumption, Storage

In [ ]:
base_url = "https://www.fueleconomy.gov/feg/"
makers = ["Chevrolet", "Ford", "Nissan"]

def get_data(base, maker):
    mpge = []
    mpg = []
    battery = []
    gas_tank = []
    for m in maker:
        page = 1
        while True:
            maker_url = f"{base}PowerSearch.do?action=noform&path=4&year1=2019&year2=2024&make={m}&srchtyp=newMake&pageno={page}&rowLimit=50"
            response = r.get(maker_url)
            soup = bs(response.content)
                
            link = [x.get("href") for x in soup.select("a[href]")[154:210] if "yourMpgVehicle" not in x.get("href")]
    
            for l in link:
                try:
                    response = r.get(base+l)
                    soup = bs(response.content)
                    type = soup.find("td", class_="fuel nobottomborder padding").getText(strip=True)
                except:
                    print(l)
                
                if type == "Electricity":
                    battery.append(soup.find_all("td", class_="even sbsCellData")[2].getText().split(" ")[0]) #hrs per 240v
                    gas_tank.append(np.nan)
                    mpge.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                    mpg.append(np.nan)
                elif type in ["Regular Gasoline", "Diesel"]:
                    battery.append(np.nan)
                    gas_tank.append(soup.find_all("td", class_="sbsEconData")[2].getText().split(" ")[0]) #gallons
                    mpge.append(np.nan)
                    mpg.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                else:
                    try:
                        battery.append(soup.find_all("td", class_="even sbsCellData")[2].getText().split(" ")[0]) #hrs per 240v
                    except:
                        battery.append(np.nan)
                    try:
                        gas_tank.append(soup.find_all("td", class_="sbsEconData")[3].getText().split(" ")[0]) #gallons
                    except:
                        gas_tank.append(np.nan)
                    mpge.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                    mpg.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
            page += 1
        else:
            break
    print("Creating Dictionary.")
    manufacturer_dict = {"electric_consumption":mpge, "electric_storage":battery,
                        "analogue_consumption":mpg, "analogue_storage":gas_tank}
    print("Returning Dataframe.")
    return pd.DataFrame(manufacturer_dict)

manufacturer_df = get_data(base_url, makers)
print("Finished")

In [145]:
manufacturer_df.head(30)

,electric_consumption,electric_storage,analogue_consumption,analogue_storage
0,120,7.5,NaN,NaN
1,120,7.5,NaN,NaN
2,119,9.3,NaN,NaN
3,118,10,NaN,NaN
4,118,10,NaN,NaN
5,115,7.5,NaN,NaN
6,115,7.5,NaN,NaN
7,96,9.5,NaN,NaN
8,106,4.5,106,8.9
9,67,11,NaN,NaN
